In [1]:
import glc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dir(glc)

['AdjustForC13',
 'ESIData',
 'ESISearchPatterns',
 'ESISpecies',
 'EdgeDf',
 'EmbStats',
 'FeatDicts',
 'FeatMapper',
 'FeatureESI',
 'GGM',
 'GLCModel',
 'LoadExampleData',
 'MzPeakLookup',
 'PickPeaks',
 'TqdmWarning',
 'UMAPEmbStats',
 'UMAPEmbedder',
 'UMAPPlots',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'data_loading',
 'filter_by_logp',
 'glc_model',
 'ion_processor',
 'load_ggm',
 'load_lm_database',
 'logp_filtering',
 'map_ungrouped_feature_table',
 'mz_peak_picking',
 'umap_model_and_plots',
 'warnings']

In [2]:
ds = glc.LoadExampleData()
feat_df = ds.feat_table
feat_dicts = glc.FeatDicts(feat_df)

ground_truth = ds.ground_truth

ggm_df = ds.ggm
ggm = glc.GGM(ggm_df)

4858
All nodes
4849


In [3]:
peak_picker = glc.PickPeaks(ggm, feat_dicts, rt_delta=2, only_pos_pcor=False)
peak_picker.get_peak_df(n_thresh=20).sort_values(by='n_edges', ascending=False)

,mz,n_edges,width,mzmin,mzmax,sigma,density,mz_peak_id
11,1.003999,1782,0.012472,0.997870,1.010342,0.002693,5.288509,11
4,0.369873,663,0.045424,0.344589,0.390013,0.008553,0.690852,4
7,0.634732,580,0.044684,0.614741,0.659425,0.010486,0.610847,7
20,2.008627,401,0.015796,2.001444,2.017240,0.003657,0.974137,20
32,4.956601,323,0.010968,4.951242,4.962211,0.002382,0.975613,32
...,...,...,...,...,...,...,...,...
85,41.997411,21,0.015022,41.989359,42.004381,0.002960,0.046811,85
25,2.947366,21,0.006614,2.943292,2.949906,0.001079,0.064708,25
88,76.049955,20,0.008336,76.046212,76.054548,0.001950,0.057346,88
40,16.587868,20,0.009885,16.583656,16.593541,0.002409,0.074714,40


In [4]:
esi_data = {
    'formula': [
        'H',
        'Na',
        'K',
        'NH4',
        'H2O',
        'C2H2',
        'H2'

    ], 
    'type_code': [
        'charged',
        'charged',
        'charged',
        'charged',
        'isf',
        'isf',
        'isf'
    ]
}
esi_df = pd.DataFrame(esi_data)



In [5]:
feat2lmids = glc.map_ungrouped_feature_table(
    db= glc.load_lm_database(), 
    feat_dicts=feat_dicts,
    ion_mode='pos', 
    proton_only=False, 
    esi_df=esi_df,
    ppm_interval=10, 
    mz_peak_picker_obj=peak_picker, 
    c13_adjustment=True, 
    logp_filter_tresh=3
)

/Users/tomrix/Library/CloudStorage/OneDrive-ImperialCollegeLondon/projects/glc/src/glc/data_loading.py:62: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  lm_df = pd.read_csv(f)
/Users/tomrix/Library/CloudStorage/OneDrive-ImperialCollegeLondon/projects/glc/src/glc/ion_processor.py:174: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dfs, axis=0).reset_index(drop=True)
Adjusting for C13 isotope components: 100%|██████████| 1439/1439 [00:00<00:00, 5199.84it/s]


In [ ]:
# feat2lmids = glc.map_ungrouped_feature_table(
#     db= glc.load_lm_database(), 
#     feat_dicts=feat_dicts,
#     ion_mode='pos', 
#     proton_only=True, 
    
# )

In [6]:
model = glc.GLCModel(
    ggm=ggm,
    feat_dicts=feat_dicts,
    node_ids= feat2lmids, 
    db_df= glc.load_lm_database(),
)
predictions = model.predict_all()

/Users/tomrix/Library/CloudStorage/OneDrive-ImperialCollegeLondon/projects/glc/src/glc/data_loading.py:62: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  lm_df = pd.read_csv(f)
Predicting feature classes: 100%|██████████| 4849/4849 [00:37<00:00, 130.61it/s]


In [12]:
annot_df = ds.ground_truth
print(annot_df.columns)
annot_df 

Index(['peak_id', 'cpdID', 'cpdName', 'Feature Name', 'lm_id', 'lm_category',
       'lm_mainclass', 'lm_subclass', 'm/z', 'Retention Time', 'ion', 'IonID',
       'Monoisotopic_mass', 'chemicalFormula', 'logPRDKit', 'InChIKey',
       'InChI', 'MSIAnnotationLevel'],
      dtype='object')


,peak_id,cpdID,cpdName,Feature Name,lm_id,lm_category,lm_mainclass,lm_subclass,m/z,Retention Time,ion,IonID,Monoisotopic_mass,chemicalFormula,logPRDKit,InChIKey,InChI,MSIAnnotationLevel
0,173,LPOS-0012,Tetradecanoylcarnitine CAR(14:0),CAR(14:0),LMFA07070107,Fatty Acyls [FA],Fatty esters [FA07],Fatty acyl carnitines [FA0707],372.308734,78.457216,[M+H]+,LPOS-0012.1,371.303559,C21H41NO4,-1.5975,PSHXNVGSVNEJBD-LJQANCHMSA-N,InChI=1S/C21H41NO4/c1-5-6-7-8-9-10-11-12-13-14...,2
1,150,LPOS-0014,Tetradecenoylcarnitine CAR(14:1),CAR(14:1),na,Fatty Acyls [FA],Fatty esters [FA07],Fatty acyl carnitines [FA0707],370.293109,65.830733,[M+H]+,LPOS-0014.1,369.287909,C21H39NO4,na,na,na,1
2,225,LPOS-0016,Palmitoylcarnitine CAR(16:0),CAR(16:0),LMFA07070004,Fatty Acyls [FA],Fatty esters [FA07],Fatty acyl carnitines [FA0707],400.339722,109.137232,[M+H]+,LPOS-0016.1,399.334859,C23H45NO4,-1.3093,XOMRRQXKHMYMOC-OAQYLSRUSA-N,InChI=1S/C23H45NO4/c1-5-6-7-8-9-10-11-12-13-14...,1
3,272,LPOS-0021,Octadecenoylcarnitine CAR(18:1),CAR(18:1),na,Fatty Acyls [FA],Fatty esters [FA07],Fatty acyl carnitines [FA0707],426.355716,118.764734,[M+H]+,LPOS-0021.1,425.350509,C25H47NO4,na,na,na,1
4,269,LPOS-0023,Octadecadienoylcarnitine CAR(18:2),CAR(18:2),na,Fatty Acyls [FA],Fatty esters [FA07],Fatty acyl carnitines [FA0707],424.340134,97.297885,[M+H]+,LPOS-0023.1,423.334859,C25H45NO4,na,na,na,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,4679,LPOS-0363,TG(60:3),TG(60:3),na,Glycerolipids [GL],Triradylglycerols [GL03],Triacylglycerols [GL0301],1007.840912,700.278236,[M+K]+,LPOS-0363.1,968.877191,C63H116O6,na,na,na,2
179,963,LPOS-0380,Luteine,Luteine,LMPR01070274,Prenol Lipids [PR],Isoprenoids [PR01],C40 isoprenoids (tetraterpenes) [PR0107],568.426056,228.695662,[M+],LPOS-0380.1,568.428030,C40H56O2,na,KBPHJBAIARWVSC-RGZFRNHPSA-N,InChI=1S/C40H56O2/c1-29(17-13-19-31(3)21-23-37...,1
180,65,LPOS-0383,Retinol,retinol,LMPR01090001,Prenol Lipids [PR],Isoprenoids [PR01],Retinoids [PR0109],286.226887,141.746229,[M+],LPOS-0383.1,286.229670,C20H30O,2.211,FPIPGXGPPPQFEQ-OVSJKPMPSA-N,InChI=1S/C20H30O/c1-16(8-6-9-17(2)13-15-21)11-...,1
181,965,LPOS-0384,Zeaxanthin,zeaxanthin,LMPR01070261,Prenol Lipids [PR],Isoprenoids [PR01],C40 isoprenoids (tetraterpenes) [PR0107],568.426061,219.430146,[M+],LPOS-0384.1,568.428030,C40H56O2,4.4408,JKQXZKUSFCKOGQ-QAYBQHTQSA-N,InChI=1S/C40H56O2/c1-29(17-13-19-31(3)21-23-37...,1


In [18]:
from typing import Dict, List, Tuple
import pandas as pd 


def test_annotations(predictions: Dict[int, List[Tuple[str, float]]], annot_df: pd.DataFrame, annot_clf_col: str = 'lm_subclass', k: int = 10) -> pd.DataFrame:
    """
    Scores annotations based on the predicted class

    Parameters:
    - predictions (dict): Dictionary of predictions for each feature from ggm_classify.ClassPredictor.predict_all().
    - annot_df (pd.DataFrame): DataFrame containing the annotations.
    - annot_clf_col (str): Column name containing the annotated class
    - k (int): Number of top predictions to consider to get pcor_based quality score

    Returns:
    - pd.DataFrame: DataFrame containing the scored annotations.
    """

    def number_of_top_preds(preds):
        """"""
        return sum(1 for _, score in preds if score == preds[0][1])

    results = []
    for feature, chem_class in zip(annot_df['peak_id'].astype(int), annot_df[annot_clf_col]):
        try:
            preds = predictions[feature]

            if len(preds) > 0:
                topk = preds[:k]  
                scores = [i[1] for i in topk]  
                k1_frac = scores[0] / sum(scores) 

                
                results.append({
                    'feature': feature,
                    'annot_class': chem_class,
                    'pred_class': preds[0][0],
                    'pred_score': preds[0][1], 
                    'fraction': k1_frac,
                    'n_top': number_of_top_preds(preds)
                })
            else:
                print(f'Feature: {feature} failed')
        except Exception as e:
            # Usually the feature was not in the main-subgraph and so not used in the GGM
            print(f'Feature: {feature} can not be computed as it is not in the main subgraph of the GGM.')

    df = pd.DataFrame(results)
    df['correct'] = df['annot_class'] == df['pred_class'] 
    return df


x = test_annotations(predictions, annot_df, k=10)#['correct'].sum()/183

In [ ]:
from typing import Dict, List, Tuple, Optional
import pandas as pd


def test_annotations(
    predictions: Dict[int, List[Tuple[str, float]]],
    annot_df: pd.DataFrame,
    annot_clf_col: str = "lm_subclass",
    verbose: bool = True,
) -> pd.DataFrame:
    """
    Evaluate annotation quality against predicted classes.

    Args:
        predictions: Dictionary mapping feature_id -> list of (class, score) tuples. Intended to be ```glc.GLCModel.predict_all()``` output.
        annot_df: DataFrame with true annotations and must include a 'peak_id' column.
        annot_clf_col: Column name containing the annotated class. (default: 'lm_subclass')
        verbose: If True, prints warnings for missing features. These should be annotations not in the GGM main subgraph, which GLC cannot predict.

    Returns:
        A DataFrame comparing predictions to annotations. 
    """

    results = []

    for feature, true_class in zip(
        annot_df["peak_id"].astype(int), annot_df[annot_clf_col]
    ):
        # Prediction retrieval
        pred_list = predictions.get(feature)

        if not pred_list:
            if verbose:
                print(
                    f"Feature {feature} skipped (missing or not in GGM main subgraph)."
                )
            continue

        # Top prediction
        top_class, top_score = pred_list[0]

        # Count equally highest scores
        n_top = sum(1 for _, s in pred_list if s == top_score)

        results.append(
            {
                "feature": feature,
                "annot_class": true_class,
                "pred_class": top_class,
                "pred_score": top_score,
                "n_top": n_top,
            }
        )

    # Build output DataFrame
    df = pd.DataFrame(results)
    if not df.empty:
        df["correct"] = df["annot_class"] == df["pred_class"]
    else:
        df["correct"] = []

    return df

df_eval = test_annotations(predictions, annot_df)
accuracy = df_eval["correct"].mean()
accuracy


np.float64(0.8524590163934426)

In [23]:
df_eval

,feature,annot_class,pred_class,pred_score,n_top,correct
0,173,Fatty acyl carnitines [FA0707],Fatty acyl carnitines [FA0707],1.030083,1,True
1,150,Fatty acyl carnitines [FA0707],Fatty acyl carnitines [FA0707],1.003142,1,True
2,225,Fatty acyl carnitines [FA0707],Fatty acyl carnitines [FA0707],0.751271,1,True
3,272,Fatty acyl carnitines [FA0707],Fatty acyl carnitines [FA0707],0.798729,1,True
4,269,Fatty acyl carnitines [FA0707],Fatty acyl carnitines [FA0707],0.690414,1,True
...,...,...,...,...,...,...
178,4679,Triacylglycerols [GL0301],Triacylglycerols [GL0301],0.752061,1,True
179,963,C40 isoprenoids (tetraterpenes) [PR0107],C40 isoprenoids (tetraterpenes) [PR0107],0.425898,1,True
180,65,Retinoids [PR0109],Monoacylglycerophosphocholines [GP0105],0.176466,1,False
181,965,C40 isoprenoids (tetraterpenes) [PR0107],C40 isoprenoids (tetraterpenes) [PR0107],0.234797,1,True
